In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-filtered-final-books/ratings_final_filtered.csv
/kaggle/input/amazon-filtered-final-books/books_final.csv


In [2]:
ratings = pd.read_csv('/kaggle/input/amazon-filtered-final-books/ratings_final_filtered.csv')
books = pd.read_csv('/kaggle/input/amazon-filtered-final-books/ratings_final_filtered.csv')
ratings.head()

,book_id,rating,User_id
0,172246,4.0,0
1,172246,5.0,1
2,172246,5.0,2
3,172246,5.0,3
4,172246,5.0,4


In [3]:
ratings.rename(columns={'User_id':'user',"book_id":"item","rating":"label"},inplace=True)
ratings = ratings.iloc[:,[2,0,1]]
ratings.head()

,user,item,label
0,0,172246,4.0
1,1,172246,5.0
2,2,172246,5.0
3,3,172246,5.0
4,4,172246,5.0


In [4]:
# Initialize filtered_ratings with the original DataFrame
filtered_ratings = ratings.copy()

while True:
    # Count ratings for each item
    item_counts = filtered_ratings['item'].value_counts()
    
    # Count ratings for each user
    user_counts = filtered_ratings['user'].value_counts()
    
    # Check if there are any items with less than 5 ratings
    items_with_less_than_5 = (item_counts < 5).sum()
    
    # Check if there are any users with less than 5 ratings
    users_with_less_than_11 = (user_counts < 11).sum()
    
    # If no items or users with less than 5 ratings, stop the loop
    if items_with_less_than_5 == 0 and users_with_less_than_11 == 0:
        break
    
    # Filter out items with less than 5 ratings
    items_to_keep = item_counts[item_counts >= 5].index
    filtered_ratings = filtered_ratings[filtered_ratings['item'].isin(items_to_keep)]
    
    # Filter out users who rated less than 5 items
    user_counts = filtered_ratings['user'].value_counts()  # Recalculate after filtering items
    users_to_keep = user_counts[user_counts >= 11].index
    filtered_ratings = filtered_ratings[filtered_ratings['user'].isin(users_to_keep)]
    
    # Print intermediate results
    print(f"Items with less than 5 ratings: {items_with_less_than_5}")
    print(f"Users with less than 11 ratings: {users_with_less_than_11}")
    print(f"Filtered ratings shape: {filtered_ratings.shape}")

# Final result
print("Final shape of filtered ratings:", filtered_ratings.shape)

Items with less than 5 ratings: 0
Users with less than 11 ratings: 616657
Filtered ratings shape: (270250, 3)
Items with less than 5 ratings: 14394
Users with less than 11 ratings: 0
Filtered ratings shape: (228731, 3)
Items with less than 5 ratings: 685
Users with less than 11 ratings: 0
Filtered ratings shape: (225038, 3)
Items with less than 5 ratings: 102
Users with less than 11 ratings: 0
Filtered ratings shape: (224432, 3)
Items with less than 5 ratings: 14
Users with less than 11 ratings: 0
Filtered ratings shape: (224366, 3)
Items with less than 5 ratings: 2
Users with less than 11 ratings: 0
Filtered ratings shape: (224358, 3)
Final shape of filtered ratings: (224358, 3)


In [5]:
!pip install -U LibRecommender

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 43.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tsfresh 0.20.3 requires scipy>=1.14.0; python_version >= "3.10", but you ha

In [121]:
from libreco.algorithms import LightGCN  # pure data, algorithm LightGCN
from libreco.data import DatasetPure, DataInfo,DatasetFeat
from libreco.data import (
    random_split,
    split_by_num,
    split_by_num_chrono,
    split_by_ratio,
    split_by_ratio_chrono,
)
from libreco.evaluation import evaluate

In [163]:
train_data_df, eval_data_df = split_by_ratio(filtered_ratings, test_size=0.4)

In [123]:
train_data, data_info = DatasetPure.build_trainset(train_data_df)
eval_data = DatasetPure.build_evalset(eval_data_df)
print(data_info)  # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

n_users: 7591, n_items: 8846, data density: 0.2008 %


In [124]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="bpr",
    embed_size=768,
    n_epochs=3,
    lr=0.01,
#     batch_size=128,
    num_neg = 1,
    sampler = 'popular',
    device="cuda",
)

In [125]:
# monitor metrics on eval_data during training
lightgcn.fit(
    train_data,
    neg_sampling=True,  # sample negative items for train and eval data
    verbose=2,
#     eval_batch_size=128,
    k=5,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

Training start time: 2024-10-16 12:42:39


train: 100%|██████████| 527/527 [00:26<00:00, 19.73it/s]


Epoch 1 elapsed: 26.715s
	 train_loss: 0.2174


eval_listwise: 100%|██████████| 7306/7306 [00:09<00:00, 746.30it/s]


	 eval log_loss: 0.6334
	 eval roc_auc: 0.7878
	 eval precision@5: 0.1057
	 eval recall@5: 0.1434
	 eval ndcg@5: 0.3083


...

	 eval log_loss: 0.6798
	 eval roc_auc: 0.7844
	 eval precision@5: 0.1081
	 eval recall@5: 0.1443
	 eval ndcg@5: 0.3179


train: 100%|██████████| 527/527 [00:26<00:00, 19.68it/s]


Epoch 3 elapsed: 26.777s
	 train_loss: 0.097


eval_listwise: 100%|██████████| 7306/7306 [00:09<00:00, 740.19it/s]


	 eval log_loss: 0.7330
	 eval roc_auc: 0.7747
	 eval precision@5: 0.0987
	 eval recall@5: 0.1318
	 eval ndcg@5: 0.2964


In [90]:
item_embeddings = lightgcn.get_item_embedding()
item_embeddings

array([[ 0.09963668,  0.08864461,  0.00112784, ...,  0.09645559,
        -0.0733121 ,  0.08811449],
       [ 0.07118866,  0.00516627,  0.14089805, ..., -0.00478089,
         0.1265871 , -0.15710299],
       [ 0.17084062,  0.07073134, -0.05280448, ..., -0.0796074 ,
         0.01552415, -0.1510223 ],
       ...,
       [ 0.09647404,  0.00710135,  0.07753615, ...,  0.01337749,
        -0.0043819 , -0.34843323],
       [-0.16051859, -0.2035412 , -0.03229182, ..., -0.19405633,
        -0.14068781,  0.03336784],
       [-0.02802931,  0.21064623, -0.01654552, ..., -0.1227415 ,
        -0.14234672, -0.01882999]], dtype=float32)

In [91]:
user_embeddings = lightgcn.get_user_embedding()
user_embeddings

array([[-0.08385067, -0.24153233,  0.35537177, ...,  0.10798094,
         0.04733916, -0.26839572],
       [-0.23116657, -0.15692657,  0.31471738, ...,  0.11050634,
         0.06683128, -0.06715871],
       [-0.15347467,  0.07102732,  0.16859427, ..., -0.20992936,
        -0.09147061, -0.0133398 ],
       ...,
       [-0.09342539,  0.0722834 , -0.18902262, ..., -0.00482304,
        -0.09935858, -0.07886828],
       [-0.2015414 , -0.22106071, -0.06066193, ...,  0.094919  ,
         0.41269484,  0.07347537],
       [-0.05977311, -0.09951837,  0.18567644, ...,  0.18979922,
         0.24661116, -0.14783728]], dtype=float32)

In [93]:
filtered_ratings.head()

,user,item,label
169,164,11104,5.0
179,174,11104,5.0
189,184,11104,5.0
191,186,11104,5.0
193,188,11104,5.0


In [128]:
# predict preference of user 2211 to item 110
print("prediction: ", lightgcn.predict(user=174, item=11104))

prediction:  [0.99999607]


In [132]:
print("recommendation: ", lightgcn.recommend_user(user=174, n_rec=7))

recommendation:  {174: array([ 66911, 184154, 137063, 158751, 170038, 108147,   5885])}


In [134]:
lightgcn.search_knn_items(11104, 4)

[11104, 5241, 5239, 5249]

In [135]:
lightgcn.search_knn_users(174, 4)

[174, 1334, 74511, 34160]

In [110]:
# cold-start prediction
print(
    "cold prediction: ",
    lightgcn.predict(user="ccc", item="not item", cold_start="average"),
)
# cold-start recommendation
print(
    "cold recommendation: ",
    lightgcn.recommend_user(user="are we good?", n_rec=7, cold_start="popular"),
)

Detect 1 unknown interaction(s), position: [0]
cold prediction:  [0.551936]
Detect unknown user: are we good?
cold recommendation:  {'are we good?': array([202807,  53272,  65711,  89905, 183734, 120093,  51848])}


In [140]:
path = '/kaggle/working/new'
model_name="lightgcn"

In [147]:
data_info.save(path, model_name)
lightgcn.save(path, model_name,manual=True, inference_only=False)

In [159]:
data_info2 = DataInfo.load(path, model_name)

In [160]:
print("new data_info: ", data_info2)

new data_info:  n_users: 7591, n_items: 8846, data density: 0.2008 %


Retraining

In [166]:
import tensorflow as tf

def reset_state(name):
    tf.compat.v1.reset_default_graph()
    print("\n", "=" * 30, name, "=" * 30)
#%%
reset_state("retrain")
#%%
train_data, data_info3 = DatasetFeat.merge_trainset(
        eval_data_df, data_info2, merge_behavior=True)


 ============================== retrain ==============================


In [167]:
print("new data_info: ", data_info3)

new data_info:  n_users: 7591, n_items: 8846, data density: 0.0627 %


In [168]:
lightgcn2 = LightGCN(
    task="ranking",
    data_info=data_info3,
    loss_type="bpr",
    embed_size=768,
    n_epochs=3,
    lr=0.01,
#     batch_size=128,
    num_neg = 1,
    sampler = 'popular',
    device="cuda",
)

In [171]:
lightgcn2.rebuild_model(path, model_name)

KeyError: 0

In [ ]:
# monitor metrics on eval_data during training
lightgcn.fit(
    train_data,
    neg_sampling=True,  # sample negative items for train and eval data
    verbose=2,
#     eval_batch_size=128,
    k=5,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)